#### Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import warnings
import sys
import pandas as pd
import numpy as np
import torch
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle

sys.path.append("../")
from pipeline import data
from pipeline.config import CONF
from pipeline.data import plots
from pipeline.data import io
from pipeline.data import inspection
from pipeline.data import preprocess
from pipeline.models import hyperopt
from pipeline.models import training
from pipeline.models.dataset import TimeSeriesDataset
from torch.utils.data import DataLoader
from pipeline.models.transformer import TimeSeriesTransformer
import pickle
from tqdm import tqdm

# To suppress all warnings
warnings.filterwarnings("ignore")

# black is a code formatter (see https://github.com/psf/black).
# It will automatically format the code you write in the cells imposing consistent Python style.
%load_ext jupyter_black
# matplotlib style file
# Template for style file: https://matplotlib.org/stable/tutorials/introductory/customizing.html#customizing-with-style-sheets
plt.style.use("../matplotlib_style.txt")
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load raw data

This takes about 1 minute for the first time.

In [ ]:
if CONF.pipeline.process_raw_data and not CONF.data.loaded_raw_data:
    # Load raw data
    (
        Installed_Capacity_Germany_Raw,
        Prices_Europe_Raw,
        Realised_Supply_Germany_Raw,
        Realised_Demand_Germany_Raw,
        Weather_Data_Germany_Raw,
        Weather_Data_Germany_2022_Raw,
    ) = data.load_data(CONF=CONF)
    CONF.data.loaded_raw_data = True

In [ ]:
if CONF.pipeline.process_raw_data:
    Installed_Capacity_Germany = Installed_Capacity_Germany_Raw.copy()
    Prices_Europe = Prices_Europe_Raw.copy()
    Realised_Supply_Germany = Realised_Supply_Germany_Raw.copy()
    Realised_Demand_Germany = Realised_Demand_Germany_Raw.copy()
    Weather_Data_Germany = Weather_Data_Germany_Raw.copy()
    Weather_Data_Germany_2022 = Weather_Data_Germany_2022_Raw.copy()

### Inspect raw data

##### Inspect missingness

In [ ]:
if CONF.pipeline.process_raw_data and CONF.pipeline.inspect:
    data.save_data_inspection(
        Installed_Capacity_Germany=Installed_Capacity_Germany,
        Prices_Europe=Prices_Europe,
        Realised_Supply_Germany=Realised_Supply_Germany,
        Realised_Demand_Germany=Realised_Demand_Germany,
        Weather_Data_Germany=Weather_Data_Germany,
        Weather_Data_Germany_2022=Weather_Data_Germany_2022,
        CONF=CONF,
        data_type="raw",
    )

##### Inspect resolution

In [ ]:
if CONF.pipeline.process_raw_data:
    inspection.date_range_and_resolution_dfs(
        Installed_Capacity_Germany=Installed_Capacity_Germany,
        Prices_Europe=Prices_Europe,
        Realised_Supply_Germany=Realised_Supply_Germany,
        Realised_Demand_Germany=Realised_Demand_Germany,
        Weather_Data_Germany=Weather_Data_Germany,
    )

#### Plot raw data

In [ ]:
if CONF.pipeline.plot:
    plots.plot_df(
        Installed_Capacity_Germany,
        "Installed_Capacity_Germany",
        CONF,
        processed_data=False,
    )
    plots.plot_df(Prices_Europe, "Prices_Europe", CONF, processed_data=False)
    plots.plot_df(
        Realised_Supply_Germany, "Realised_Supply_Germany", CONF, processed_data=False
    )
    plots.plot_df(
        Realised_Demand_Germany, "Realised_Demand_Germany", CONF, processed_data=False
    )
    plots.plot_df(
        Weather_Data_Germany,
        "Weather_Data_Germany",
        CONF,
        date_col=io.DATE_COLUMNS_WEATHER[-1],
        drop_date_cols=io.DATE_COLUMNS_WEATHER,
        processed_data=False,
    )

### Raw data pipeline

##### Merging weather data frames

In [ ]:
if CONF.pipeline.process_raw_data:
    # Remove the data for 2022 from the original dataframe
    Weather_Data_Germany = Weather_Data_Germany[
        Weather_Data_Germany["time"].dt.year != 2022
    ]

    # Concatenate the filtered original dataframe with the 2022 data
    Weather_Data_Germany = pd.concat(
        [Weather_Data_Germany, Weather_Data_Germany_2022], ignore_index=True
    )

##### Fill NaN

In [ ]:
if CONF.pipeline.process_raw_data:
    Processed_Installed_Capacity_Germany = data.process_na_values(
        Installed_Capacity_Germany, CONF
    )
    Processed_Prices_Europe = data.process_na_values(Prices_Europe, CONF)
    Processed_Realised_Supply_Germany = data.process_na_values(
        Realised_Supply_Germany, CONF
    )
    Processed_Realised_Demand_Germany = data.process_na_values(
        Realised_Demand_Germany, CONF
    )
    Processed_Weather_Data_Germany = data.process_na_values(Weather_Data_Germany, CONF)

##### Convert to Greenwich time

In [ ]:
if CONF.pipeline.process_raw_data:
    Processed_Realised_Supply_Germany = preprocess.german2greenwich(
        Processed_Realised_Supply_Germany
    )
    Processed_Realised_Demand_Germany = preprocess.german2greenwich(
        Processed_Realised_Demand_Germany
    )

##### Aggregate weather data

In [ ]:
if CONF.pipeline.process_raw_data:
    Processed_Weather_Data_Germany = preprocess.aggregate_weather_data(
        Processed_Weather_Data_Germany, ["forecast_origin", "time"]
    )

##### Decrease demand and supply's time resolution

In [ ]:
if CONF.pipeline.process_raw_data:
    Processed_Realised_Demand_Germany = Processed_Realised_Demand_Germany[
        Processed_Realised_Demand_Germany["Date to"].dt.minute == 0
    ]
    Processed_Realised_Demand_Germany["Date from"] = Processed_Realised_Demand_Germany[
        "Date to"
    ] - pd.Timedelta(hours=1)
    Processed_Realised_Supply_Germany = Processed_Realised_Supply_Germany[
        Processed_Realised_Supply_Germany["Date to"].dt.minute == 0
    ]
    Processed_Realised_Supply_Germany["Date from"] = Processed_Realised_Supply_Germany[
        "Date to"
    ] - pd.Timedelta(hours=1)

##### Increase time resolution of capacity

In [ ]:
if CONF.pipeline.process_raw_data:
    new_row = {
        "Date from": pd.Timestamp("2022-12-31 23:00:00"),
        "Date to": pd.Timestamp("2023-01-01 00:00:00"),
    }
    new_row_df = pd.DataFrame([new_row])
    Processed_Installed_Capacity_Germany_hourly = pd.concat(
        [Processed_Installed_Capacity_Germany, new_row_df], ignore_index=True
    )

    Processed_Installed_Capacity_Germany_hourly = (
        Processed_Installed_Capacity_Germany_hourly.set_index("Date from")
    )
    Processed_Installed_Capacity_Germany_hourly = (
        Processed_Installed_Capacity_Germany_hourly.resample("H").mean()
    )
    Processed_Installed_Capacity_Germany_hourly.reset_index(inplace=True)
    Processed_Installed_Capacity_Germany_hourly["Date to"] = (
        Processed_Installed_Capacity_Germany_hourly["Date from"] + pd.Timedelta(hours=1)
    )
    Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany_hourly
    Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany.fillna(
        method="ffill"
    )
    inspection.date_range_and_resolution(
        Processed_Installed_Capacity_Germany, io.DATE_COLUMNS
    )

##### Trim rows of every df to have same range

In [ ]:
if CONF.pipeline.process_raw_data:
    # trim first row of Processed_Weather_Data_Germany
    Processed_Weather_Data_Germany = Processed_Weather_Data_Germany[
        Processed_Weather_Data_Germany["time"]
        != Processed_Weather_Data_Germany["time"].min()
    ]

    # trim last row of every other df
    Processed_Installed_Capacity_Germany = Processed_Installed_Capacity_Germany[
        Processed_Installed_Capacity_Germany["Date to"]
        != Processed_Installed_Capacity_Germany["Date to"].max()
    ]
    Processed_Prices_Europe = Processed_Prices_Europe[
        Processed_Prices_Europe["Date to"] != Processed_Prices_Europe["Date to"].max()
    ]
    Processed_Realised_Supply_Germany = Processed_Realised_Supply_Germany[
        Processed_Realised_Supply_Germany["Date to"]
        != Processed_Realised_Supply_Germany["Date to"].max()
    ]
    Processed_Realised_Demand_Germany = Processed_Realised_Demand_Germany[
        Processed_Realised_Demand_Germany["Date to"]
        != Processed_Realised_Demand_Germany["Date to"].max()
    ]

##### Patch time saving

In [ ]:
if CONF.pipeline.process_raw_data:
    Processed_Prices_Europe = preprocess.patch_time_saving(Processed_Prices_Europe)
    Processed_Realised_Demand_Germany = preprocess.patch_time_saving(
        Processed_Realised_Demand_Germany
    )
    Processed_Realised_Supply_Germany = preprocess.patch_time_saving(
        Processed_Realised_Supply_Germany
    )

##### Normalize data

In [ ]:
if CONF.pipeline.process_raw_data and CONF.pipeline.normalize_data:
    print("Split data in train, val and test")
    (
        Processed_Installed_Capacity_Germany,
        Processed_Prices_Europe,
        Processed_Realised_Supply_Germany,
        Processed_Realised_Demand_Germany,
        Processed_Weather_Data_Germany,
    ) = preprocess.split_dfs(
        Processed_Installed_Capacity_Germany,
        Processed_Prices_Europe,
        Processed_Realised_Supply_Germany,
        Processed_Realised_Demand_Germany,
        Processed_Weather_Data_Germany,
    )

    print("Normalizing data")
    (
        Processed_Installed_Capacity_Germany,
        Processed_Installed_Capacity_Germany_Scalers,
    ) = preprocess.normalize_data(
        df=Processed_Installed_Capacity_Germany,
        ignore_features=io.DATE_COLUMNS,
        constant=CONF.data.price_normalization_constant,
    )

    Processed_Prices_Europe, Processed_Prices_Europe_Scalers = (
        preprocess.normalize_data(
            df=Processed_Prices_Europe,
            ignore_features=io.DATE_COLUMNS,
            constant=CONF.data.price_normalization_constant,
        )
    )
    Processed_Realised_Supply_Germany, Processed_Realised_Supply_Germany_Scalers = (
        preprocess.normalize_data(
            df=Processed_Realised_Supply_Germany, ignore_features=io.DATE_COLUMNS
        )
    )
    Processed_Realised_Demand_Germany, Processed_Realised_Demand_Germany_Scalers = (
        preprocess.normalize_data(
            df=Processed_Realised_Demand_Germany, ignore_features=io.DATE_COLUMNS
        )
    )
    Processed_Weather_Data_Germany, Processed_Weather_Data_Germany_Scalers = (
        preprocess.normalize_data(
            df=Processed_Weather_Data_Germany,
            ignore_features=io.DATE_COLUMNS_WEATHER + ["longitude", "latitude"],
        )
    )

    print("Remove train, test, val columns, again")
    Processed_Installed_Capacity_Germany = preprocess.remove_train_val_test_cols(
        Processed_Installed_Capacity_Germany
    )
    Processed_Prices_Europe = preprocess.remove_train_val_test_cols(
        Processed_Prices_Europe
    )
    Processed_Realised_Supply_Germany = preprocess.remove_train_val_test_cols(
        Processed_Realised_Supply_Germany
    )
    Processed_Realised_Demand_Germany = preprocess.remove_train_val_test_cols(
        Processed_Realised_Demand_Germany
    )
    Processed_Weather_Data_Germany = preprocess.remove_train_val_test_cols(
        Processed_Weather_Data_Germany
    )

##### Save scalers

In [ ]:
if CONF.pipeline.process_raw_data:
    # Dictionary of scalers, each associated with a dataset
    scalers = {
        "Processed_Installed_Capacity_Germany_Scalers": Processed_Installed_Capacity_Germany_Scalers,
        "Processed_Prices_Europe_Scalers": Processed_Prices_Europe_Scalers,
        "Processed_Realised_Supply_Germany_Scalers": Processed_Realised_Supply_Germany_Scalers,
        "Processed_Realised_Demand_Germany_Scalers": Processed_Realised_Demand_Germany_Scalers,
        "Processed_Weather_Data_Germany_Scalers": Processed_Weather_Data_Germany_Scalers,
    }

    # Ensure the directory exists where scalers will be stored
    scalers_dir = CONF.data.preprocessed_data_dir
    os.makedirs(scalers_dir, exist_ok=True)

    # Serialize and save each scaler to a file
    for scaler_name, scaler in scalers.items():
        scaler_path = os.path.join(scalers_dir, f"{scaler_name}.pkl")
        with open(scaler_path, "wb") as file:
            pickle.dump(scaler, file)
        print(f"Scaler stored: {scaler_path}")

##### Merge data into a single frame and store everything

In [ ]:
if CONF.pipeline.process_raw_data:

    def add_prefix(df, prefix):
        """
        Add a prefix to all column names except the merge key.
        """
        return df.rename(
            columns={
                col: prefix + col if col not in io.DATE_COLUMNS else col
                for col in df.columns
            }
        )

    Suffix_Processed_Prices_Europe = add_prefix(Processed_Prices_Europe, "prices_")
    Suffix_Processed_Installed_Capacity_Germany = add_prefix(
        Processed_Installed_Capacity_Germany, "capacity_"
    )
    Suffix_Processed_Realised_Supply_Germany = add_prefix(
        Processed_Realised_Supply_Germany, "supply_"
    )
    Suffix_Processed_Realised_Demand_Germany = add_prefix(
        Processed_Realised_Demand_Germany, "demand_"
    )
    Suffix_Processed_Weather_Data_Germany = add_prefix(
        Processed_Weather_Data_Germany, "weather_"
    )
    Suffix_Processed_Weather_Data_Germany = (
        Suffix_Processed_Weather_Data_Germany.rename(
            columns={"weather_time": "Date to"}
        )
    )

    # Now perform the merge
    df = pd.merge(
        Suffix_Processed_Prices_Europe,
        Suffix_Processed_Installed_Capacity_Germany,
        on=io.DATE_COLUMNS,
        how="inner",
    )
    df = pd.merge(
        df, Suffix_Processed_Realised_Supply_Germany, on=io.DATE_COLUMNS, how="inner"
    )
    df = pd.merge(
        df, Suffix_Processed_Realised_Demand_Germany, on=io.DATE_COLUMNS, how="inner"
    )
    df = pd.merge(
        df, Suffix_Processed_Weather_Data_Germany, on=io.DATE_COLUMNS[-1], how="inner"
    )
    df = preprocess.split_data(df=df, column_name=io.DATE_COLUMNS[-1])
    df.to_csv(os.path.join(CONF.data.preprocessed_data_dir, "df.csv"), index=False)
    df.head()

## Data test

##### Inspect missingness

In [ ]:
# Generate profile reports

if CONF.pipeline.data_test and CONF.pipeline.inspect:
    data.save_data_inspection(
        Installed_Capacity_Germany=Processed_Installed_Capacity_Germany,
        Prices_Europe=Processed_Prices_Europe,
        Realised_Supply_Germany=Processed_Realised_Supply_Germany,
        Realised_Demand_Germany=Processed_Realised_Demand_Germany,
        Weather_Data_Germany=Processed_Weather_Data_Germany,
        CONF=CONF,
        data_type="preprocessed",
    )

##### Plot processed data

In [ ]:
if CONF.pipeline.data_test and CONF.pipeline.plot:
    plots.plot_df(
        Processed_Installed_Capacity_Germany, "Installed_Capacity_Germany", CONF
    )
    plots.plot_df(Processed_Prices_Europe, "Prices_Europe", CONF)
    plots.plot_df(Processed_Realised_Supply_Germany, "Realised_Supply_Germany", CONF)
    plots.plot_df(Processed_Realised_Demand_Germany, "Realised_Demand_Germany", CONF)
    plots.plot_df(
        Processed_Weather_Data_Germany,
        "Weather_Data_Germany",
        CONF,
        date_col=io.DATE_COLUMNS_WEATHER[-1],
        drop_date_cols=io.DATE_COLUMNS_WEATHER,
    )

##### Inspect time's resolution

In [ ]:
if CONF.pipeline.data_test:
    inspection.date_range_and_resolution_dfs(
        Installed_Capacity_Germany=Processed_Installed_Capacity_Germany,
        Prices_Europe=Processed_Prices_Europe,
        Realised_Supply_Germany=Processed_Realised_Supply_Germany,
        Realised_Demand_Germany=Processed_Realised_Demand_Germany,
        Weather_Data_Germany=Processed_Weather_Data_Germany,
        processed=True,
    )

##### Data Unit Tests

In [ ]:
if CONF.pipeline.data_test:
    dfs = [
        Processed_Installed_Capacity_Germany,
        Processed_Prices_Europe,
        Processed_Realised_Supply_Germany,
        Processed_Realised_Demand_Germany,
    ]

    raw_dfs = [
        Installed_Capacity_Germany_Raw,
        Prices_Europe_Raw,
        Realised_Supply_Germany_Raw,
        Realised_Demand_Germany_Raw,
    ]

    # assert all data frames have same length.
    def test_dataframe_lengths(dfs):
        expected_length = len(dfs[0])
        for df in dfs[1:]:
            assert len(df) == expected_length, "DataFrames have different lengths"

    test_dataframe_lengths(dfs)

    # assert all data frames have have same time resolution
    def test_dataframe_resolutions(dfs):
        date_column = "Date from"
        expected_resolution = dfs[0][date_column].diff().dropna().mode()[0]
        for df in dfs[1:]:
            current_resolution = df[date_column].diff().dropna().mode()[0]
            assert (
                current_resolution == expected_resolution
            ), "DataFrames have different time resolutions"
        current_resolution = (
            Processed_Weather_Data_Germany["time"].diff().dropna().mode()[0]
        )
        assert (
            current_resolution == expected_resolution
        ), "DataFrames have different time resolutions"

    test_dataframe_resolutions(dfs)

    # Assert every row of every df has the same "Date to"
    def test_date_to_consistency(dfs):
        date_to_values = dfs[0]["Date to"].values
        for i, df in enumerate(dfs[1:]):
            assert all(
                df["Date to"].values == date_to_values
            ), f"Mismatch in 'Date to' values across DataFrames {i + 1}"
        assert np.all(
            Processed_Weather_Data_Germany["time"].values == date_to_values
        ), "DataFrames have different time resolutions"

    test_date_to_consistency(dfs)

    # assert that not cell is missed
    def test_no_missing_cells(dfs):
        for i, df in enumerate(dfs + [Processed_Weather_Data_Germany]):
            for column in df.columns:
                assert (
                    df[column].isnull().sum() == 0
                ), f"Missing values found in column '{column}' of DataFrame at index {i}"

    test_no_missing_cells(dfs)

    # assert that raw dfs and normal dfs have same set of columns
    def test_same_columns(raw_dfs, processed_dfs, ignore_columns=None):
        if ignore_columns is None:
            ignore_columns = set()

        for raw_df, processed_df in zip(raw_dfs, processed_dfs):
            raw_columns = set(raw_df.columns) - ignore_columns
            processed_columns = set(processed_df.columns) - ignore_columns

            assert (
                raw_columns == processed_columns
            ), f"Column mismatch between raw and processed DataFrames. Missing {raw_columns - processed_columns} or {processed_columns - raw_columns}"

    # Example usage of the function, ignoring 'train', 'val', and 'test'
    test_same_columns(
        raw_dfs,
        dfs,
        ignore_columns={
            "∅ Neighbouring DE/LU [€/MWh]",
            "Hungary [€/MWh]",
            "Poland [€/MWh]",
            "DE/AT/LU [€/MWh]",
        },
    )

##### Sanity test plot final dataframe

In [ ]:
if CONF.pipeline.data_test and CONF.pipeline.plot:
    plots.plot_df(df, "Final Dataframe", CONF, figsize=(150, 30))

## Hyperparameters optimization

In [ ]:
df = pd.read_csv(os.path.join(CONF.data.preprocessed_data_dir, "df.csv"))

In [ ]:
if CONF.pipeline.do_hyperopt:
    hyperopt.hyper_parameter_optimize()

## Train with best hyperparameters on train, val

In [ ]:
if CONF.pipeline.do_final_train:
    best_hyperparameters = pickle.load(open(CONF.model.best_hyperparameter_path, "rb"))
    training.train_loop(best_hyperparameters, df, "best", merge_train_val=True)

## Test Evaluation

##### Inference test set

In [ ]:
best_hyperparameters = pickle.load(open(CONF.model.best_hyperparameter_path, "rb"))

if CONF.pipeline.do_test:
    model = TimeSeriesTransformer(hyperparameters=best_hyperparameters).to(
        device
    )  # Adjust parameters as necessary
    model = model.to(device)
    model_state_dict = torch.load(CONF.model.final_model_path)
    model.load_state_dict(model_state_dict)
    model.eval()
    test_loss = 0
    predictions = []
    ground_truths = []
    test_df = df[df["test"]].reset_index()
    test_dataset = TimeSeriesDataset(test_df, hyperparameters=best_hyperparameters)
    test_loader = DataLoader(
        test_dataset, batch_size=best_hyperparameters.train.batch_size, shuffle=False
    )

    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            ground_truths.append(targets.cpu().numpy())
            predictions.append(outputs.cpu().numpy())
    ground_truths = np.concatenate(ground_truths)
    predictions = np.concatenate(predictions)

##### Scale predictions and ground truth back to original scale

In [ ]:
if CONF.pipeline.do_test:
    # Define the path to the scaler file
    scaler_path = os.path.join(
        best_hyperparameters.data.preprocessed_data_dir,
        "Processed_Realised_Supply_Germany_Scalers.pkl",
    )

    # Load the scaler from the file
    with open(scaler_path, "rb") as file:
        Processed_Realised_Supply_Germany_Scalers = pickle.load(file)

    # Invert the normalization
    for tidx, target in enumerate(best_hyperparameters.model.targets):
        scaler = Processed_Realised_Supply_Germany_Scalers[target.split("_")[-1]]
        predictions[:, :, tidx] = scaler.inverse_transform(predictions[:, :, tidx])
        ground_truths[:, :, tidx] = scaler.inverse_transform(ground_truths[:, :, tidx])

##### Compute test loss on scaled data

In [ ]:
criterion = best_hyperparameters.train.loss()
losses = []
with torch.no_grad():
    for preds, targets in tqdm(zip(predictions, ground_truths)):
        loss = criterion(torch.tensor(preds), torch.tensor(targets))
        losses.append(loss.item())
print(f"Mean Squared Error: {np.mean(losses)}")

##### Visualize predictions

In [ ]:
# Assuming CONF.test.do_test is checked somewhere else in your code
if CONF.pipeline.do_test:
    _, num_horizons, num_targets = ground_truths.shape

    # Create a figure with subplots arranged in 2 rows and 7 columns
    fig, axes = plt.subplots(
        num_horizons, num_targets, figsize=(54, 18)
    )  # Adjust the figsize to ensure adequate spacing

    # Loop over each horizon and target combination
    for h in range(num_horizons):
        for t in range(num_targets):
            ax = axes[h, t]  # Locate the right subplot
            ax.plot(
                ground_truths[:, h, t], label="Ground Truth"
            )  # Plot ground truth for horizon h, target t
            ax.plot(
                predictions[:, h, t], label="Predictions"
            )  # Plot predictions for horizon h, target t
            ax.set_title(
                f"Horizon {best_hyperparameters.horizons[h]}, Target {CONF.model.targets[t]}"
            )
            ax.set_xlabel("Time")
            ax.set_ylabel("Value")
            ax.legend()

    fig.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the plot

##### Visualize residuals

In [ ]:
if CONF.pipeline.do_test:
    _, num_horizons, num_targets = ground_truths.shape

    # Create a figure with subplots arranged in 2 rows and 7 columns
    fig, axes = plt.subplots(
        num_horizons, num_targets, figsize=(54, 18)
    )  # Adjust the figsize to ensure adequate spacing

    # Loop over each horizon and target combination
    for h in range(num_horizons):
        for t in range(num_targets):
            ax = axes[h, t]  # Locate the right subplot
            residuals = (
                predictions[:, h, t] - ground_truths[:, h, t]
            )  # Calculate residuals
            ax.hist(
                residuals,
                bins=30,
                alpha=0.75,
                color="blue",  # Choose bin count and style
            )
            ax.set_title(
                f"Residuals Histogram for Horizon {best_hyperparameters.horizons[h]}, Target {CONF.model.targets[t]}"
            )
            ax.set_xlabel("Residuals")
            ax.set_ylabel("Frequency")
            ax.grid(True)  # Optional: Adds grid for better readability

    fig.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the plot